In [ ]:
import whisper
import couchdb
import os
from tinytag import TinyTag
import time
import subprocess
model = whisper.load_model('small')
import gc
for limpio in range(3):
    gc.collect(generation=limpio)

In [ ]:
couch = couchdb.Server('')#Ingresar la ubicación del gestor en el equipo.
dbname = "testimonios"
if dbname in couch:
    db = couch[dbname]
else:
    db = couch.create(dbname)

In [ ]:
original = os.path.abspath(os.getcwd())
path = os.path.join(original, 'fracciones')
contenido = os.listdir(path)
contenido.sort()

In [ ]:
nombre = '' #Es el nombre del sujeto al que se le hará la transcripción
etapa = '' #Momento de la grabación
frases = []
duracion = 0
for grabacion in contenido:
    inicio = time.time()
    ruta = os.path.join(path, grabacion)
    result = model.transcribe(ruta)
    frases.append(result['text'])
    tiempo = TinyTag.get(ruta)
    minutos = tiempo.duration
    duracion += minutos 
    final = time.time()
    tiempo_trans = (final - inicio)/60
    print('el audio {} que dura {} minutos, se transcribio en {} minutos'.format(grabacion, minutos/60, tiempo_trans))

audio = " ".join(frases)
documento = {'nombre': nombre, 'etapa': etapa, 'duracion':duracion, 'relato' : audio}
db.save(documento)
for limpio in range(3):
    gc.collect(generation=limpio)
subprocess.run('shutdown -h 2', shell=True)